In [4]:
%pip install --upgrade pip
%pip install boto3 prettytable construct

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [5]:
import boto3
import subprocess
import json
import uuid
import construct as c
import websocket
import re
import os
import time
from prettytable import PrettyTable
from collections import defaultdict
from concurrent.futures import ThreadPoolExecutor, as_completed

import sys
sys.path.append('..')
from awsutils import *

# Get nginx_status from nginx stub_status module

In [6]:
def session_reader(session: dict) -> str:
    AgentMessageHeader = c.Struct(
        "HeaderLength" / c.Int32ub,
        "MessageType" / c.PaddedString(32, "ascii"),
    )

    AgentMessagePayload = c.Struct(
        "PayloadLength" / c.Int32ub,
        "Payload" / c.PaddedString(c.this.PayloadLength, "ascii"),
    )

    connection = websocket.create_connection(session["streamUrl"])
    try:
        init_payload = {
            "MessageSchemaVersion": "1.0",
            "RequestId": str(uuid.uuid4()),
            "TokenValue": session["tokenValue"],
        }
        connection.send(json.dumps(init_payload))
        while True:
            resp = connection.recv()
            message = AgentMessageHeader.parse(resp)
            if "channel_closed" in message.MessageType:
                raise Exception(
                    "Channel closed before command output was received")
            if "output_stream_data" in message.MessageType:
                break
    finally:
        connection.close()
    payload_message = AgentMessagePayload.parse(resp[message.HeaderLength:])
    return payload_message.Payload


def execute_curl_on_task(ecs_client, cluster_name, task_arn, container_name):
    # Command to execute
    command = "curl http://localhost:2000/nginx_status"

    # Start ECS Exec session
    exec_resp = ecs_client.execute_command(
        cluster=cluster_name,
        task=task_arn,
        container=container_name,
        command=command,
        interactive=True
    )

    response = session_reader(exec_resp["session"])
    return response


def parse_curl_output(output):
    # Splitting the output into lines
    lines = output.strip().split('\n')

    # Extracting the values
    active = int(re.search(r'Active connections: (\d+)', lines[0]).group(1))
    accepts, handled, requests = map(int, re.findall(r'(\d+)', lines[2]))
    reading, writing, waiting = map(int, re.findall(
        r'Reading: (\d+) Writing: (\d+) Waiting: (\d+)', lines[3])[0])

    return {
        'active': active,
        'accepts': accepts,
        'handled': handled,
        'requests': requests,
        'reading': reading,
        'writing': writing,
        'waiting': waiting
    }


def get_nginx_status(profile, region_name, cluster_name, service_name):
    aws_session = set_aws_credentials(profile, region_name=region_name)

    # Create ECS client using the session
    ecs_client = aws_session.client('ecs')

    # List all running tasks in a specified service
    task_arns = get_tasks(ecs_client, cluster_name, service_name)

    print(f"Found {len(task_arns)} tasks running in {profile}/{region_name}")

    container_name = 'nginx'

    # Initialize a defaultdict for summing up the values
    aggregated_stats = defaultdict(int)
    aggregated_stats['instance_count'] = len(task_arns)

    '''
    for task_arn in task_arns:
        response = execute_curl_on_task(ecs_client, cluster_name, task_arn, container_name)
        stats = parse_curl_output(response)

        print(f"Task {task_arn} - {stats}")
        # Iterate over each dictionary and sum up the values
        for key, value in stats.items():
            aggregated_stats[key] += value
    '''

    # Use ThreadPoolExecutor to execute tasks in parallel
    with ThreadPoolExecutor(max_workers=8) as executor:
        # Submit all tasks to the executor
        future_to_task = {executor.submit(
            execute_curl_on_task, ecs_client, cluster_name, task_arn, container_name): task_arn for task_arn in task_arns}

        for future in as_completed(future_to_task):
            task_arn = future_to_task[future]
            try:
                response = future.result()
                stats = parse_curl_output(response)
                #print(f"Task {task_arn} - {stats}")
                # Sum up the values
                for key, value in stats.items():
                    aggregated_stats[key] += value
            except Exception as e:
                print(f"Task {task_arn} failed: {e}")

    unicorn_workers = 2
    handling = aggregated_stats["active"] - aggregated_stats["instance_count"] * unicorn_workers
    aggregated_stats["queue_avg"] = round(float(handling) / aggregated_stats["instance_count"],1) if handling > 0 else 0
    return aggregated_stats



# Specify your cluster and service name
cluster_name = 'EcsCluster1-main'
service_name = 'projects-application-fargate-main'
#service_name = 'results-application-fargate-main'

environments = [
    ('acl-playground', 'us-west-2'),  # playground
    ('acl-staging', 'us-west-2'),  # staging
    ('acl-staging', 'us-east-1'),  # preprod?
    ('acl-production', 'us-east-1'),
    ('acl-production', 'eu-central-1'),
    ('acl-production', 'af-south-1'),
    ('acl-production', 'ca-central-1'),
    ('acl-production', 'ap-northeast-1'),
    ('acl-production', 'ap-southeast-1'),
    ('acl-production', 'ap-southeast-2'),
    ('acl-production', 'sa-east-1'),
]

table = PrettyTable()
table.field_names = ["Profile", "Region", "tasks", "active", "handled",
                     "accepts", "requests", "reading", "writing", "waiting", "queue_avg"]

for profile, region in environments:
    aggregated_stats = get_nginx_status(profile, region, cluster_name, service_name)
    table.add_row([profile, region,
                   aggregated_stats['instance_count'],
                   aggregated_stats['active'],
                   aggregated_stats['handled'],
                   aggregated_stats['accepts'],
                   aggregated_stats['requests'],
                   aggregated_stats['reading'],
                   aggregated_stats['writing'],
                   aggregated_stats['waiting'], 
                   aggregated_stats['queue_avg']
                   ])

table.reversesort = True
table.sortby = "requests"
print(table)

Found 1 tasks running in acl-playground/us-west-2
Found 4 tasks running in acl-staging/us-west-2
Found 3 tasks running in acl-staging/us-east-1
Found 20 tasks running in acl-production/us-east-1
Found 29 tasks running in acl-production/eu-central-1
Found 4 tasks running in acl-production/af-south-1
Task arn:aws:ecs:af-south-1:707785685172:task/EcsCluster1-main/3377bd6db96d4e3ba323ed4fb58edfce failed: Channel closed before command output was received
Found 4 tasks running in acl-production/ca-central-1
Found 4 tasks running in acl-production/ap-northeast-1
Found 6 tasks running in acl-production/ap-southeast-1
Found 6 tasks running in acl-production/ap-southeast-2
Found 6 tasks running in acl-production/sa-east-1
+----------------+----------------+-------+--------+---------+---------+----------+---------+---------+---------+-----------+
|    Profile     |     Region     | tasks | active | handled | accepts | requests | reading | writing | waiting | queue_avg |
+----------------+--------

`Active connections` - The current number of active client connections including Waiting connections.

`accepts` - The total number of accepted client connections.

`handled` - The total number of handled connections. Generally, the parameter value is the same as accepts unless some resource limits have been reached (for example, the worker_connections limit).

`requests` - The total number of client requests.

`Reading` - The current number of connections where nginx is reading the request header.

`Writing` - The current number of connections where nginx is writing the response back to the client.

`Waiting` - The current number of idle client connections waiting for a request.

`avg_queue` - The average number of requests waiting in the queue per task. This is calculated as (`active` - `workers`) / `tasks`.

# Print task parameters

In [7]:
def get_service_params(profile, region_name, cluster_name, service_name):
    aws_session = set_aws_credentials(profile, region_name=region_name)

    # Create ECS client using the session
    ecs_client = aws_session.client('ecs')

    # List all running tasks in a specified service
    task_arns = get_tasks(ecs_client, cluster_name, service_name)

    print(f"Found {len(task_arns)} tasks running in {profile}/{region_name}")

    task_definition = get_task_definition(ecs_client, cluster_name, task_arns)

    container_definition = get_container_definition(task_definition, container_name='application')

    environment_variables = container_definition.get('environment', [])


    vcpu = round(float(task_definition['cpu']) / 1024, 1)
    memory = int(task_definition['memory']) // 1024
    unicorn_workers = next((env_var['value'] for env_var in environment_variables if env_var['name'] == 'UNICORN_WORKERS'), None)
    
    return vcpu, memory, unicorn_workers

    

# Specify your cluster and service name
cluster_name = 'EcsCluster1-main'
service_name = 'projects-application-fargate-main'

environments = [
    ('acl-playground', 'us-west-2'),  # playground
    ('acl-staging', 'us-west-2'),  # staging
    ('acl-staging', 'us-east-1'),  # preprod?
    ('acl-production', 'us-east-1'),
    ('acl-production', 'eu-central-1'),
    ('acl-production', 'af-south-1'),
    ('acl-production', 'ca-central-1'),
    ('acl-production', 'ap-northeast-1'),
    ('acl-production', 'ap-southeast-1'),
    ('acl-production', 'ap-southeast-2'),
    ('acl-production', 'sa-east-1'),
]


table = PrettyTable()
table.field_names = ["Profile", "Region", "vCPU", "Memory", "Unicorn Workers"]


for profile, region in environments:
    vcpu, memory, unicorn_workers = get_service_params(profile, region, cluster_name, service_name)
    table.add_row([profile, region, vcpu, f"{memory} GB", unicorn_workers])

print(table)


Found 1 tasks running in acl-playground/us-west-2
Found 4 tasks running in acl-staging/us-west-2
Found 3 tasks running in acl-staging/us-east-1
Found 20 tasks running in acl-production/us-east-1
Found 29 tasks running in acl-production/eu-central-1
Found 4 tasks running in acl-production/af-south-1
Found 4 tasks running in acl-production/ca-central-1
Found 4 tasks running in acl-production/ap-northeast-1
Found 6 tasks running in acl-production/ap-southeast-1
Found 6 tasks running in acl-production/ap-southeast-2
Found 6 tasks running in acl-production/sa-east-1
+----------------+----------------+------+--------+-----------------+
|    Profile     |     Region     | vCPU | Memory | Unicorn Workers |
+----------------+----------------+------+--------+-----------------+
| acl-playground |   us-west-2    | 0.5  |  3 GB  |        2        |
|  acl-staging   |   us-west-2    | 0.5  |  3 GB  |        4        |
|  acl-staging   |   us-east-1    | 1.0  |  5 GB  |        4        |
| acl-produc